In [1]:
import os
import tensorflow as tf

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

if tf.test.gpu_device_name() != "/device:GPU:0":
    print("WARNING: GPU device not found.")
else:
    print("SUCCESS: Found GPU: {}".format(tf.test.gpu_device_name()))

import sys

# Append root path
sys.path.append("../")
from data.preprocessing import dataset_preprocessing
import pickle

from model.mixed_effects import *
from utils.utils import *
from utils.fe_models import get_model

In [2]:
import sys

# Append root path
sys.path.append("../")
from data.preprocessing import dataset_preprocessing
import pickle

dataset_name = "churn"
target = "binary"
mode = "cv"
RS = 42
hct = 10
test_ratio = 0.1
val_ratio = 0.1
folds = 5

# Attention: preprocessing script is meant to be executed from the notebooks directory
dataset_preprocessing.process_dataset(
    dataset_name=dataset_name,
    target=target,
    mode=mode,
    RS=RS,
    hct=hct,
    test_ratio=test_ratio,
    val_ratio=val_ratio,
    folds=folds,
)
data_path = f"{mode}_RS{RS}_hct{hct}_{folds}folds"

with open(
    f"../data/prepared/{dataset_name}/{data_path}/data_dict.pickle", "rb"
) as handle:
    data_dict = pickle.load(handle)

fold_num = 0
x_cols = data_dict[f"X_train_{fold_num}"].columns
X_train = data_dict[f"X_train_{fold_num}"]
Z_train = data_dict[f"Z_train_{fold_num}"]
y_train = data_dict[f"y_train_{fold_num}"]

X_val = data_dict[f"X_val_{fold_num}"]
Z_val = data_dict[f"Z_val_{fold_num}"]
y_val = data_dict[f"y_val_{fold_num}"]

X_test = data_dict[f"X_test_{fold_num}"]
Z_test = data_dict[f"Z_test_{fold_num}"]
y_test = data_dict[f"y_test_{fold_num}"]

qs = (
    np.max(
        [
            tf.reduce_max(Z_train, axis=0),
            tf.reduce_max(Z_val, axis=0),
            tf.reduce_max(Z_test, axis=0),
        ],
        axis=0,
    )
    + 1
)

X_train = tf.convert_to_tensor(X_train)
Z_train = tf.convert_to_tensor(Z_train, dtype=tf.int32)
y_train = tf.convert_to_tensor(y_train)

X_val = tf.convert_to_tensor(X_val)
Z_val = tf.convert_to_tensor(Z_val, dtype=tf.int32)
y_val = tf.convert_to_tensor(y_val)

X_test = tf.convert_to_tensor(X_test)
Z_test = tf.convert_to_tensor(Z_test, dtype=tf.int32)
y_test = tf.convert_to_tensor(y_test)

2025-02-13 13:21:51.652 | INFO     | data.preprocessing.dataset_preprocessing:process_dataset:61 - After processing the raw dataset churn, intermediate data kept at: ../data/prepared/churn/cv_RS42_hct10_5folds
2025-02-13 13:21:51.661 | DEBUG    | data.preprocessing.dataset_preprocessing:process_dataset:78 - Columns in raw dataset: Index(['state', 'account_length', 'area_code', 'international_plan',
       'voice_mail_plan', 'number_vmail_messages', 'total_day_minutes',
       'total_day_calls', 'total_day_charge', 'total_eve_minutes',
       'total_eve_calls', 'total_eve_charge', 'total_night_minutes',
       'total_night_calls', 'total_night_charge', 'total_intl_minutes',
       'total_intl_calls', 'total_intl_charge',
       'number_customer_service_calls', 'class'],
      dtype='object')
2025-02-13 13:21:51.662 | DEBUG    | data.preprocessing.dataset_preprocessing:process_dataset:83 - Columns in raw dataset, after dropping columns with more than 5% missings values: Index(['state', '

In [ ]:
tf.random.set_seed(RS)
np.random.seed(RS)

batch_size = 512
epochs = 500
early_stopping = 20
model_name = "AutoGluon"
loss_use = lambda: tf.keras.losses.BinaryCrossentropy


if target == "categorical":
    n_classes = np.unique(y_train).shape[0]
elif target == "binary":
    n_classes = 1

d = X_train.shape[1]  # columns
n = X_train.shape[0]  # rows
num_outputs = n_classes
perc_numeric = d / (d + Z_train.shape[1])

#             qs = np.max([tf.reduce_max(Z_train, axis=0),tf.reduce_max(Z_val, axis=0),tf.reduce_max(Z_test, axis=0)],axis=0)+1

set_seed(RS)

fe_model, optimizer = get_model(
    model_name=model_name,
    input_size=X_train.shape[1],
    output_size=num_outputs,
    target=target,
    perc_numeric=perc_numeric,
    RS=RS,
)

initial_stds = np.ones([len(qs), num_outputs]).astype(float).tolist()

me_model = MixedEffectsNetwork(
    X_train,
    Z_train,
    y_train,
    fe_model,
    target=target,
    qs=qs,
    initial_stds=initial_stds,
    fe_loss_weight=1.0,
    mode="intercepts",
    early_stopping_fe=early_stopping,
)

me_model.compile(
    loss_class_me=loss_use()(),
    loss_class_fe=loss_use()(),
    #     metric_class_me = tf.keras.metrics.AUC(multi_label=True, name="auc_me"),
    #     metric_class_fe = tf.keras.metrics.AUC(multi_label=True, name="auc_fe"),
    optimizer=optimizer,
)

mcmc = MCMCSamplingCallback(
    num_mcmc_samples=1,
    perc_burnin=0.7,
    warm_restart=None,
    num_burnin_steps=1,
    step_size=0.1,  # initial_step_size,
)


print_metric = PrintMetrics(X_train, Z_train, y_train, X_val, Z_val, y_val)

start = time.time()
history = me_model.fit(
    [X_train, Z_train],
    y_train,
    callbacks=[
        mcmc,
        print_metric,
        tf.keras.callbacks.EarlyStopping(
            monitor="me_auc_val", patience=early_stopping, mode="max"
        ),
    ],
    epochs=epochs,
    validation_data=[[X_val, Z_val], y_val],
    batch_size=batch_size,
)

end = time.time()
fit_time_gmenn = round(end - start, 2)

y_train_pred_gmenn, y_train_pred_gmenn_fe = me_model([X_train, Z_train])
y_val_pred_gmenn, y_val_pred_gmenn_fe = me_model([X_val, Z_val])
y_test_pred_gmenn, y_test_pred_gmenn_fe = me_model([X_test, Z_test])

Random seed set as 42

 Start sampling for epoch 1 of training
Epoch 1/500
8/8 [==============================] - 1s 16ms/step - me_loss: 0.7702 - me_loss_val: 0.7543 - fe_loss: 0.5995 - fe_loss_val: 0.5971 - me_auc: 0.4605 - me_auc_val: 0.4520 - fe_auc: 0.5526 - fe_auc_val: 0.5377 - stds: 0.8128 - acceptance_rate: 0.0000e+00
Adapt step size to 0.05000000074505806

 Start sampling for epoch 2 of training
Epoch 2/500
8/8 [==============================] - 0s 7ms/step - me_loss: 0.7092 - me_loss_val: 0.6947 - fe_loss: 0.5438 - fe_loss_val: 0.5427 - me_auc: 0.4719 - me_auc_val: 0.4648 - fe_auc: 0.6040 - fe_auc_val: 0.5946 - stds: 0.8128 - acceptance_rate: 0.0000e+00
Adapt step size to 0.02500000037252903

 Start sampling for epoch 3 of training
Epoch 3/500
8/8 [==============================] - 0s 8ms/step - me_loss: 0.6677 - me_loss_val: 0.6538 - fe_loss: 0.5078 - fe_loss_val: 0.5071 - me_auc: 0.4731 - me_auc_val: 0.4657 - fe_auc: 0.6179 - fe_auc_val: 0.5968 - stds: 0.8128 - acceptance_r

In [4]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test.numpy(), y_test_pred_gmenn.numpy())

0.7747307380576376